Refrência: https://github.com/samchaaa/alpaca_tech_screener/blob/master/tech_screener_notebook.ipynb

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.signal import find_peaks,argrelextrema

In [2]:
df = yf.download('PETR4.SA','2016-01-01','2020-10-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,6.570000,7.030000,6.550000,6.870000,5.179965,45962100
2016-01-05,6.920000,7.000000,6.620000,6.680000,5.036705,29446700
2016-01-06,6.530000,6.540000,6.400000,6.400000,4.825586,67507200
2016-01-07,6.190000,6.430000,6.080000,6.260000,4.720026,57387900
2016-01-08,6.380000,6.450000,6.130000,6.270000,4.727566,52100300
...,...,...,...,...,...,...
2020-09-24,20.240000,20.709999,20.059999,20.400000,16.580267,76310000
2020-09-25,20.150000,20.260000,19.870001,20.129999,16.360821,40150400
2020-09-28,20.330000,20.559999,19.629999,19.629999,15.954440,48797400


In [4]:
 df.reset_index(level=0, inplace=True)

In [5]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-04,6.570000,7.030000,6.550000,6.870000,5.179965,45962100
1,2016-01-05,6.920000,7.000000,6.620000,6.680000,5.036705,29446700
2,2016-01-06,6.530000,6.540000,6.400000,6.400000,4.825586,67507200
3,2016-01-07,6.190000,6.430000,6.080000,6.260000,4.720026,57387900
4,2016-01-08,6.380000,6.450000,6.130000,6.270000,4.727566,52100300
...,...,...,...,...,...,...,...
1180,2020-09-24,20.240000,20.709999,20.059999,20.400000,16.580267,76310000
1181,2020-09-25,20.150000,20.260000,19.870001,20.129999,16.360821,40150400
1182,2020-09-28,20.330000,20.559999,19.629999,19.629999,15.954440,48797400
1183,2020-09-29,19.629999,19.860001,19.250000,19.309999,15.694359,52878100


In [ ]:
def get_max_min(prices, smoothing, window_range):
    smooth_prices = prices['close'].rolling(window=smoothing).mean().dropna()
    local_max = argrelextrema(smooth_prices.values, np.greater)[0]
    local_min = argrelextrema(smooth_prices.values, np.less)[0]
    price_local_max_dt = []
    for i in local_max:
        if (i>window_range) and (i<len(prices)-window_range):
            price_local_max_dt.append(prices.iloc[i-window_range:i+window_range]['close'].idxmax())
    price_local_min_dt = []
    for i in local_min:
        if (i>window_range) and (i<len(prices)-window_range):
            price_local_min_dt.append(prices.iloc[i-window_range:i+window_range]['close'].idxmin())  
    maxima = pd.DataFrame(prices.loc[price_local_max_dt])
    minima = pd.DataFrame(prices.loc[price_local_min_dt])
    max_min = pd.concat([maxima, minima]).sort_index()
    max_min.index.name = 'date'
    max_min = max_min.reset_index()
    max_min = max_min[~max_min.date.duplicated()]
    p = prices.reset_index()   
    max_min['day_num'] = p[p['timestamp'].isin(max_min.date)].index.values
    max_min = max_min.set_index('day_num')['close']
    
    return max_min

In [6]:
ema_list = [3, 10, 20, 30, ]
window_list = [3, 10, 20, 30, ]

In [7]:
smooth_prices = df['Close'].rolling(window=3).mean().dropna()
local_max = argrelextrema(smooth_prices.values, np.greater)[0]
local_min = argrelextrema(smooth_prices.values, np.less)[0]

In [8]:
smooth_prices

2        6.650000
3        6.446667
4        6.310000
5        6.206667
6        5.963333
          ...    
1180    20.476666
1181    20.253333
1182    20.053333
1183    19.689999
1184    19.516666
Name: Close, Length: 1183, dtype: float64

In [9]:
price_local_max_dt = []
for i in local_max:
    if (i>3) and (i<len(df)-3): 
        price_local_max_dt.append(df.iloc[i-3:i+3]['Close'].idxmax())

In [10]:
price_local_min_dt = []
for i in local_min:
    if (i>3) and (i<len(df)-3):
        price_local_min_dt.append(df.iloc[i-3:i+3]['Close'].idxmin()) 

In [11]:
maxima = pd.DataFrame(df.loc[price_local_max_dt])
minima = pd.DataFrame(df.loc[price_local_min_dt])
max_min = pd.concat([maxima, minima]).sort_index()
max_min.index.name = 'date'
max_min = max_min.reset_index()
max_min = max_min[~max_min.date.duplicated()]

In [12]:
max_min['day_num'] = df[df['Date'].isin(max_min.date)].index.values

ValueError: Length of values (0) does not match length of index (271)

In [ ]:
max_min = max_min.set_index('day_num')['Close']

In [ ]:
df['Date'].isin(max_min.date)

In [ ]:
df[df['Date'].isin(max_min.date)].index.values

In [16]:
import numpy as np
from matplotlib import pyplot

a=np.array([10.3,2,0.9,4,5,6,7,34,2,5,25,3,-26,-20,-29],dtype=np.float)

gradients=np.diff(a)
print(gradients)


maxima_num=0
minima_num=0
max_locations=[]
min_locations=[]
count=0
for i in gradients[:-1]:
        count+=1

    if ((cmp(i,0)>0) & (cmp(gradients[count],0)<0) & (i != gradients[count])):
        maxima_num+=1
        max_locations.append(count)     

    if ((cmp(i,0)<0) & (cmp(gradients[count],0)>0) & (i != gradients[count])):
        minima_num+=1
        min_locations.append(count)


turning_points = {'maxima_number':maxima_num,'minima_number':minima_num,'maxima_locations':max_locations,'minima_locations':min_locations}  

print(turning_points)

pyplot.plot(a)
pyplot.show()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)